# Creating wrapper functions for simulating a gradient with SIPSim

* help to run multiple gradients through pipeline

## Input parameters
  * phyloseq.bulk file 
  * taxon mapping file
  * list of genomes
  * fragments simulated for all genomes
  * bulk community richness


## workflow

* Creating a community file from OTU abundances in bulk soil samples
  * phyloseq.bulk --> OTU table --> filter to sample --> community table format
* Fragment simulation
  * simulated_fragments --> parse out fragments for target OTUs 
  * simulated_fragments --> parse out fragments from random genomes to obtain richness of interest
  * combine fragment python objects
* Convert fragment lists to kde object
* Add diffusion
* Make incorp config file
* Add isotope incorporation
* Calculating BD shift from isotope incorp
* Simulating gradient fractions
* Simulating OTU table
* Simulating PCR
* Subsampling from the OTU table
* Adding count error

## Init

In [244]:
import glob
import cPickle as pickle
import copy
import re
from IPython.display import Image

In [245]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [246]:
%%R
library(ggplot2)
library(dplyr)
library(tidyr)
library(gridExtra)
library(phyloseq)

# Input variables

In [338]:
work_dir = '/home/nick/notebook/SIPSim/dev/fullCyc/frag_norm_9_2.5_-5/'
genome_index = '/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/genome_index.txt'
genome_dir = '/home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/'
primer_file = '/home/nick/notebook/SIPSim/dev/515F-806R.fna'

target_file = '/home/nick/notebook/SIPSim/dev/fullCyc/CD-HIT/target_taxa.txt'

physeq_dir = '/home/nick/notebook/fullCyc/data/MiSeq_16S/515f-806r/V4_Lib1-7/phyloseq/'
physeq_bulkCore = 'bulk-core'
bulk_day = 1

richness = 2503    # estimated total richness of sample
nprocs = 12

In [248]:
if not os.path.isdir(work_dir):
    os.makedirs(work_dir)

# Creating a community file from bulk soil sample

In [249]:
%%R -i physeq_dir -i physeq_bulkCore
# bulk core samples
F = file.path(physeq_dir, physeq_bulkCore)
physeq.bulk = readRDS(F)
physeq.bulk.m = physeq.bulk %>% sample_data
physeq.bulk

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 4950 taxa and 9 samples ]
sample_data() Sample Data:       [ 9 samples by 17 sample variables ]
tax_table()   Taxonomy Table:    [ 4950 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 4950 tips and 4949 internal nodes ]


In [250]:
%%R -i bulk_day
# just 12C-Con samples
physeq.bulk.12C = prune_samples(physeq.bulk.m$Substrate == '12C-Con' & 
                                physeq.bulk.m$Day == bulk_day, physeq.bulk) %>%
    filter_taxa(function(x) sum(x) > 0, TRUE)
physeq.bulk.12C.m = physeq.bulk.12C %>% sample_data
physeq.bulk.12C

phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 2008 taxa and 1 samples ]
sample_data() Sample Data:       [ 1 samples by 17 sample variables ]
tax_table()   Taxonomy Table:    [ 2008 taxa by 8 taxonomic ranks ]
phy_tree()    Phylogenetic Tree: [ 2008 tips and 2007 internal nodes ]


In [251]:
%%R -i work_dir -i bulk_day

physeq2comm = function(physeq){
    df.otu = physeq %>% otu_table %>% as.matrix %>% as.data.frame
    df.otu$taxon_name = rownames(df.otu)
    df.otu = gather(df.otu, 'sample', 'abundance', 1:(ncol(df.otu)-1))
    df.otu = df.otu %>%
        group_by(sample) %>%
        mutate(rel_abund_perc = abundance / sum(abundance) * 100,
               rank = row_number(-abundance)) %>%
        ungroup() %>%
        mutate(library = gsub('^', '_', sample) %>% as.factor %>% as.numeric) %>%
        select(library, taxon_name, rel_abund_perc, rank) %>%
        arrange(library, rank)
    
}

comm.bulk.12C = physeq2comm(physeq.bulk.12C)
#comm.bulk.12C %>% head

# writing out comm file
x = paste(c('comm_bulk_12C-Con.D',bulk_day, '.txt'), collapse='')
comm_file = file.path(work_dir, x)
write.table(comm.bulk.12C, comm_file, quote=FALSE, row.names=FALSE, sep='\t')
cat('File written: ', comm_file, '\n')

File written:  /home/nick/notebook/SIPSim/dev/fullCyc/frag_norm_9_2.5_-5//comm_bulk_12C-Con.D1.txt 


### Adding on target (OTU <--> genome) to comm object

In [252]:
%%R -i target_file 

df_targets = read.delim(target_file, sep='\t')
comm_bulk_12C_j = left_join(comm.bulk.12C, df_targets, c('taxon_name' = 'OTU')) %>% 
    as.data.frame %>%
    mutate(ssu_ID = ssu_ID %>% as.character,
           genome_fileID = genome_fileID %>% as.character,
           genomeID = genomeID %>% as.character,
           genome_seqID = genome_seqID %>% as.character,
           OTU_taxonomy = OTU_taxonomy %>% as.character)
comm_bulk_12C_j %>% head(n=3)

  library taxon_name rel_abund_perc rank cluster
1       1  OTU.14142       5.918292    1      NA
2       1      OTU.2       5.856579    2     134
3       1  OTU.12920       4.424833    3      NA
                                                                ssu_ID
1                                                                 <NA>
2 rRNA_AP012157_Solibacillus_silvestris_StLB046_DNA_290186-291730_DIR+
3                                                                 <NA>
                                                                                genome_fileID
1                                                                                        <NA>
2 /home/nick/notebook/SIPSim/dev/bac_genome1210/genomes/Solibacillus_silvestris_StLB046.fasta
3                                                                                        <NA>
                         genomeID                                 genome_seqID
1                            <NA>                                

In [253]:
# R to python
%Rpull comm_bulk_12C_j
comm_bulk_12C_j.head(n=3)

,library,taxon_name,rel_abund_perc,rank,cluster,ssu_ID,genome_fileID,genomeID,genome_seqID,OTU_taxonomy
0,1,OTU.14142,5.918292,1,-2147483648,NA,NA,NA,NA,NA
1,1,OTU.2,5.856579,2,134,rRNA_AP012157_Solibacillus_silvestris_StLB046_...,/home/nick/notebook/SIPSim/dev/bac_genome1210/...,Solibacillus_silvestris_StLB046,AP012157_Solibacillus_silvestris_StLB046_DNA,Bacteria:Firmicutes:Bacilli:Bacillales:Bacilla...
2,1,OTU.12920,4.424833,3,-2147483648,NA,NA,NA,NA,NA


# Genome Fragments

## Fragment simulation of dataset genomes

In [10]:
!cd $work_dir; \
    SIPSim fragments \
    $genome_index \
    --fp $genome_dir \
    --fr $primer_file \
    --fld skewed-normal,9000,2500,-5 \
    --flr None,None \
    --nf 10000 \
    --np $nprocs \
    2> ampFrags.log \
    > ampFrags.pkl        

In [353]:
# loading amplicon fragments from full genome KDE dataset
ampFragFile = os.path.join(work_dir, 'ampFrags.pkl')

ampFrag_all = []
with open(ampFragFile, 'rb') as iFH:
    ampFrag_all = pickle.load(iFH)
print 'Count of frag-GC KDEs for all genomes: {}'.format(len(ampFrag_all))    

Count of frag-GC KDEs for all genomes: 1210


## Parsing out fragments for matching OTU-genomes

In [354]:
x = comm_bulk_12C_j.loc[comm_bulk_12C_j['cluster'] >= 0, :]
x.shape

(96, 10)

In [355]:
def parse_target_ampFrags(ampFrags, comm):
    # just OTUs in community with matching genome
    comm_target = comm.loc[comm['cluster'] >= 0, :]
    msg = 'Number of OTU-genome matches: {}\n'
    sys.stderr.write(msg.format(comm_target.shape[0]))
                               
    # making an index for ampFrag list
    target_idx = {g[0]:i for i,g in enumerate(ampFrag_all)}
    msg = 'Number of genomes with simulated fragments: {}\n'
    sys.stderr.write(msg.format(len(target_idx))) 
        
    # parsing out ampFrags
    ampFrag_target = []
    stat_cnt = 0
    for count,row in comm_target.iterrows():
        stat_cnt += 1
        if stat_cnt % 10 == 0:
            sys.stderr.write('{},'.format(stat_cnt))
        tg = row['genomeID']
        try:
            idx = target_idx[tg]
        except KeyError:
            idx = None
            msg = 'Cannot find "{}" in target list'
            print msg.format(tg)
        try:
            ampFrag_target.append(copy.deepcopy(ampFrags[idx]))
        except TypeError:
            pass
        
    # renaming genome fragment KDEs for target OTUs by target OTU-ID
    for i in range(comm_target.shape[0]):
        ampFrag_target[i][0] =  comm_target.iloc[i,1]               
        
    # check that all were found
    x_len = len(ampFrag_target)
    y_len = comm_target.shape[0]
    if(x_len != y_len):
        msg = 'No matching genomes for {} OTUs!\n'.format(y_len - x_len)
        sys.stderr.write(msg)
        
    return(ampFrag_target)
    
# main    
ampFrag_target = parse_target_ampFrags(ampFrag_all, comm_bulk_12C_j)

Number of OTU-genome matches: 96
Number of genomes with simulated fragments: 1210
10,20,30,40,50,60,70,80,90,

## Parsing out fragments of random genomes for all non-matching OTUs 

In [356]:
comm_nontarget = comm_bulk_12C_j.loc[comm_bulk_12C_j['cluster'] < 0, :]
print 'Number of OTUs in community without matching genomes: {}'.format(comm_nontarget.shape[0])

Number of OTUs in community without matching genomes: 1912


In [373]:
def parse_target_non_ampFrags(ampFrags, comm, richness_needed):
    # just OTUs in community with matching genome
    comm_nontarget = comm.loc[comm['cluster'] < 0, :]
    msg = 'Number of OTUs in community without matching genomes: {}\n'
    sys.stderr.write(msg.format(comm_nontarget.shape[0]))
    
    # making an index of non-target genome fragments
    ## will select randomly from index to select genome fragments
    nt = set(comm['genomeID'])
    ampFrag_nonTarget_idx = [i for i,x in enumerate(ampFrags) if x[0] not in nt]
    pool_size = len(ampFrag_nonTarget_idx)
    msg = 'Non-target fragment pool size: {}\n'
    sys.stderr.write(msg.format(pool_size))
        
    # parsing amp frag
    ampFrag_rand = []
    if richness_needed > 0:
        # random choice index
        r_idx = range(len(ampFrag_nonTarget_idx))
        r_idx = np.random.choice(r_idx, richness_needed)
    
        # parsing out randomly selected fragments for each non-target OTU
        stat_cnt = 0
        for i in r_idx:
            stat_cnt += 1
            if stat_cnt % 10 == 0:
                sys.stderr.write('{},'.format(stat_cnt))
            ii = ampFrag_nonTarget_idx[i]
            ampFrag_rand.append(copy.deepcopy(ampFrag_all[ii]))
        
        # renaming randomly selected KDEs by random OTU_IDs
        comm_nontarget_size = comm_nontarget.shape[0]
        for i in range(richness_needed):
            ampFrag_rand[i][0] = 'OTU.r{}'.format(i)
            #ampFrag_rand[i][0] = comm_nontarget.iloc[i,1]
        
        # checking
        x_len = len(ampFrag_rand)
        if(x_len != richness_needed):
            msg = 'Richness != needed richess; non-target richness = {}'
            sys.stderr.write(msg.format(x_len))       
            
    return ampFrag_rand
    
    
# main   
## richness needed
richness_needed = richness - len(ampFrag_target)
msg = 'Number of random taxa needed to reach richness: {}\n'
sys.stderr.write(msg.format(richness_needed))

## parsing out fragments
ampFrag_rand = parse_target_non_ampFrags(ampFrag_all, comm_bulk_12C_j, richness_needed)      

Number of random taxa needed to reach richness: 2407
Number of OTUs in community without matching genomes: 1912
Non-target fragment pool size: 1122
10,20,30,40,50,60,70,80,90,100,110,120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320,330,340,350,360,370,380,390,400,410,420,430,440,450,460,470,480,490,500,510,520,530,540,550,560,570,580,590,600,610,620,630,640,650,660,670,680,690,700,710,720,730,740,750,760,770,780,790,800,810,820,830,840,850,860,870,880,890,900,910,920,930,940,950,960,970,980,990,1000,1010,1020,1030,1040,1050,1060,1070,1080,1090,1100,1110,1120,1130,1140,1150,1160,1170,1180,1190,1200,1210,1220,1230,1240,1250,1260,1270,1280,1290,1300,1310,1320,1330,1340,1350,1360,1370,1380,1390,1400,1410,1420,1430,1440,1450,1460,1470,1480,1490,1500,1510,1520,1530,1540,1550,1560,1570,1580,1590,1600,1610,1620,1630,1640,1650,1660,1670,1680,1690,1700,1710,1720,1730,1740,1750,1760,1770,1780,1790,1800,1810,1820,1830,1840,1850,1860,1870,1880,1890,1900,1910,1920,

## Combining into 1 fragment distribution dataset

In [375]:
# final check
if len(ampFrag_target) + len(ampFrag_rand) != richness:
    msg = 'WARNING: incorrect richness!\n'
    sys.stderr.write(msg)

In [376]:
# appending random taxa to target taxa and writing
outFile = os.path.join(work_dir, 'ampFrags_wRand.pkl')

with open(outFile, 'wb') as oFH:
    x = ampFrag_target + ampFrag_rand
    print 'Number of taxa in output: {}'.format(len(x))
    pickle.dump(x, oFH)
    x = ()

Number of taxa in output: 2503


### Making a comm file for all OTUs

In [377]:
### remove large ampFrag objects from memory
ampFrag_all = ()
ampFrag_target = ()
ampFrag_rand = ()

In [340]:
#-- debug --#
ampFragFile = os.path.join(work_dir, 'ampFrags_wRand.pkl')

ampFrag_debug = []
with open(ampFragFile, 'rb') as iFH:
    ampFrag_debug = pickle.load(iFH)
print 'Count of frag-GC KDEs for all genomes: {}'.format(len(ampFrag_debug))    

Count of frag-GC KDEs for all genomes: 1210


In [378]:
#sorted([x[0] for x ina ampFrag_debug], reverse=True)

# Converting fragments to kde object

In [379]:
!cd $work_dir; \
    SIPSim fragment_kde \
    ampFrags_wRand.pkl \
    > ampFrags_wRand_kde.pkl

# Adding diffusion

In [380]:
!cd $work_dir; \
    SIPSim diffusion \
    ampFrags_wRand_kde.pkl \
    --np $nprocs \
    > ampFrags_wRand_kde_dif.pkl    

Processing: OTU.r275
Processing: OTU.r1412
Processing: OTU.r1742
Processing: OTU.r820
Processing: OTU.r274
Processing: OTU.r2053
Processing: OTU.r1411
Processing: OTU.1
Processing: OTU.r1741
Processing: OTU.r1990
Processing: OTU.r821
Processing: OTU.r2063
Processing: OTU.r2062
Processing: OTU.r277
Processing: OTU.r2050
Processing: OTU.r2051
Processing: OTU.r1696
Processing: OTU.r1410
Processing: OTU.r1030
Processing: OTU.3
Processing: OTU.r1740
Processing: OTU.r2275
Processing: OTU.r1991
Processing: OTU.r822
Processing: OTU.r1516
Processing: OTU.r1517
Processing: OTU.r2061
Processing: OTU.r276
Processing: OTU.r442
Processing: OTU.r1691
Processing: OTU.r1417
Processing: OTU.r1033
Processing: OTU.2
Processing: OTU.r1747
Processing: OTU.r1949
Processing: OTU.r1996
Processing: OTU.r823
Processing: OTU.r2060
Processing: OTU.r1514
Processing: OTU.r1825
Processing: OTU.r271
Processing: OTU.r443
Processing: OTU.r440
Processing: OTU.r1690
Processing: OTU.r1416
Processing: OTU.r1032
Processing: 

# Making an incorp config file

* no isotope incorporation: 12C-Con sample

In [381]:
!cd $work_dir; \
    SIPSim incorpConfigExample \
    --percTaxa 0 \
    --percIncorpUnif 0 \
    > PT0_PI0.config

# Adding isotope incorporation to BD distribution

In [384]:
comm_file = os.path.join(work_dir, 'comm_bulk_12C-Con.D{}.txt'.format(bulk_day))

!cd $work_dir; \
    SIPSim isotope_incorp \
    ampFrags_wRand_kde_dif.pkl \
    PT0_PI0.config \
    --comm $comm_file \
    --np $nprocs \
    > ampFrags_wRand_kde_dif_incorp.pkl

Traceback (most recent call last):
  File "/home/nick/notebook/SIPSim/bin/isotope_incorp.py", line 140, in <module>
    IsoIncorp.main(args)
  File "/home/nick/notebook/SIPSim/bin/../lib/IsoIncorp.py", line 47, in main
    _add_comm_to_kde(KDE_BD, comm)
  File "/home/nick/notebook/SIPSim/bin/../lib/IsoIncorp.py", line 177, in _add_comm_to_kde
    d['abundances'][libID] = abund[0]
IndexError: list index out of range


# Calculating BD shift from isotope incorporation

In [ ]:
print 'skipping; probably not needed'
# !cd $work_dir; \
#     SIPSim BD_shift \
#     ampFrags_wRand_kde_dif.pkl \
#     ampFrags_wRand_kde_dif_incorp.pkl \
#     --np $nprocs \
#     > ampFrags_wRand_kde_dif_incorp_BD-shift.txt